In [ ]:
import JSON
using Plots
using HDF5
include("tools.jl")
include("get_grid.jl")
include("get_params.jl")

In [ ]:
border = import_border("data/border.json")
dx = 20
Nx, Ny, xrange, yrange, isgrid, isborder, n = get_grid(border, Float64(dx))

x = repeat(reshape(xrange,1,Nx),Ny,1)
y = repeat(reshape(yrange,Ny,1),1,Nx)
x_for_bx = repeat(reshape(xrange,1,Nx),Ny,1) .- dx/2
y_for_bx = repeat(reshape(yrange,Ny,1),1,Nx)
x_for_by = repeat(reshape(xrange,1,Nx),Ny,1)
y_for_by = repeat(reshape(yrange,Ny,1),1,Nx) .+ dx/2;


In [ ]:
recompute_param = true
sigma_l = 100.
if(recompute_param)
    bx, by, p, m, d = get_params(isborder, sigma_l, "data/pantagruel.h5", "grid_params_20_100.h5")
else
    bx, by, p, m, d = get_params("grid_params_20.h5")
end


In [ ]:
interval = 200
Niter = 20000
th = zeros(Ny, Nx)

@time begin
    for k in 1:Niter
        dth = 0
        for i in 2:Ny-1
            for j in 2:Nx-1
                if(isgrid[i,j])
                    temp = th[i,j]
                    bij = (by[i-1,j] + by[i,j] + bx[i,j] + bx[i,j+1])
                    (by[i,j] * th[i+1,j] + by[i-1,j] * th[i-1,j] +
                        bx[i,j+1] * th[i,j+1] + bx[i,j] * th[i,j-1])
                    th[i,j] = (by[i,j] * th[i+1,j] + by[i-1,j] * th[i-1,j] + 
                        bx[i,j+1] * th[i,j+1] + bx[i,j] * th[i,j-1] + dx^2*p[i,j]) / bij
                    dth = max(abs(th[i,j] - temp), dth)
                end
            end
        end   
        if(mod(k,interval) == 0)
            println([k, dth])
        end
    end
end


In [ ]:
#contour(1:Nx,1:Ny,th, fill=true,levels=-2:0.2:1.6, title="Continuous", colorbar_title = "θ(x,t)")
do_plot(isgrid, th)
#do_plot(isgrid, bx)
